데이터 전처리

In [1]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, Input, Flatten, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

import sentencepiece as spm
import os

# 네이버 영화 리뷰 불러오기
data_path = os.path.dirname(os.path.abspath('__file__')) + r'/'
train_data = pd.read_table(data_path + 'ratings_train.txt')
test_data = pd.read_table(data_path + 'ratings_test.txt')

In [2]:
print('리뷰 개수 :',len(train_data)) # 리뷰 개수 출력
print('리뷰 개수 :',len(test_data))

리뷰 개수 : 150000
리뷰 개수 : 50000


In [3]:
train_data['document'].nunique(), train_data['label'].nunique()

(146182, 2)

In [4]:
test_data['document'].nunique(), test_data['label'].nunique()

(49157, 2)

In [5]:
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data.drop_duplicates(subset=['document'], inplace=True)

In [6]:
print('리뷰 개수 :',len(train_data)) # 리뷰 개수 출력
print('리뷰 개수 :',len(test_data))

리뷰 개수 : 146183
리뷰 개수 : 49158


In [7]:
print(train_data.groupby('label').size().reset_index(name = 'count'))

   label  count
0      0  73342
1      1  72841


In [8]:
train_data[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 .!?]","")
# 한글과 공백을 제외하고 모두 제거 + 온점 + ! + ?

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [10]:
train_data[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

id            0
document    406
label         0
dtype: int64


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [12]:
train_data = train_data.dropna(how = 'any')
print(len(train_data))

145777


In [13]:
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 .!?]","")
# 한글과 공백을 제외하고 모두 제거 + 온점 + ! + ?
test_data['document'] = test_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
test_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

id          0
document    0
label       0
dtype: int64


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인
test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(test_data.isnull().values.any()) # Null 값이 존재하는지 확인

False
False


토큰화

In [16]:
df = pd.concat([train_data, test_data])
df.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [17]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['document']))
    #샘플을 naver_review.txt 파일에 저장한 후에 센텐스피스를 통해 단어 집합을 생성
with open('naver_review_train.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_data['document']))
with open('naver_review_test.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(test_data['document']))

In [18]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=18000 --model_type=bpe --max_sentence_length=9999')
#vocab 생성이 완료되면 naver.model, naver.vocab 파일 두개가 생성 됩니다. .vocab 에서 학습된 subwords를 확인할 수 있습니다.

In [19]:
#spm.SentencePieceTrainer.Train('--input=naver_review_train.txt --model_prefix=naver_review_train --vocab_size=18000 --model_type=bpe --max_sentence_length=9999')
#vocab 생성이 완료되면 naver_review_train.model, naver_review_train.vocab 파일 두개가 생성 됩니다. .vocab 에서 학습된 subwords를 확인할 수 있습니다.

In [20]:
#spm.SentencePieceTrainer.Train('--input=naver_review_test.txt --model_prefix=naver_review_test --vocab_size=18000 --model_type=bpe --max_sentence_length=9999')
#vocab 생성이 완료되면 naver_review_test.model, naver_review_test.vocab 파일 두개가 생성 됩니다. .vocab 에서 학습된 subwords를 확인할 수 있습니다.

In [21]:
import csv
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
#train_vocab_list = pd.read_csv('naver_review_train.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
#test_vocab_list = pd.read_csv('naver_review_test.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
print("됨")

됨


In [22]:
#train_vocab_list

In [23]:
#test_vocab_list

In [24]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [25]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[130, 945, 1250, 2634, 16461]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 207, 806, 86]



In [26]:
sp.GetPieceSize()

18000

In [27]:
sp.IdToPiece(4)

'영화'

In [28]:
sp.PieceToId('영화')

4

In [29]:
sp.DecodeIds([54, 207, 806, 86])

'진짜 최고의 영화입니다 ᄏᄏ'

In [33]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])
x_train = np.array(train_data['document'])
x_test = np.array(test_data['document'])

In [38]:
import tensorflow as tf

In [41]:
def sp_tokenize(s, corpus):

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./naver.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({idx:word})
        index_word.update({word:idx})

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word

In [65]:
my_corpus = x_train
tensor, word_index, index_word = sp_tokenize(sp, my_corpus)
x_train = tensor
print(x_train)

[[    7  1030     3 ...     0     0     0]
 [ 1611     8  4907 ...     0     0     0]
 [   23 16495     0 ...     0     0     0]
 ...
 [  244   130 16490 ...     0     0     0]
 [ 2007   563  5265 ...     0     0     0]
 [  227     5 10842 ...     0     0     0]]


In [66]:
my_corpus = x_test
tensor, word_index, index_word = sp_tokenize(sp, my_corpus)
x_test = tensor
print(x_test)

[[ 1215   445     0 ...     0     0     0]
 [ 1375     6    70 ...     0     0     0]
 [ 2611 16464  5574 ...     0     0     0]
 ...
 [16173   821  1915 ...     0     0     0]
 [  778  1408 16464 ...     0     0     0]
 [ 5053   319   254 ...     0     0     0]]


In [53]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [69]:
model = Sequential()
model.add(Embedding(18000, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [70]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [72]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
1944/1944 [==============================] - 64s 31ms/step - loss: 0.6935 - acc: 0.5009 - val_loss: 0.6932 - val_acc: 0.4958

Epoch 00001: val_acc improved from -inf to 0.49582, saving model to best_model.h5
Epoch 2/15
1944/1944 [==============================] - 54s 28ms/step - loss: 0.6932 - acc: 0.5020 - val_loss: 0.6931 - val_acc: 0.5042

Epoch 00002: val_acc improved from 0.49582 to 0.50418, saving model to best_model.h5
Epoch 3/15
1944/1944 [==============================] - 55s 28ms/step - loss: 0.6932 - acc: 0.4961 - val_loss: 0.6935 - val_acc: 0.4958

Epoch 00003: val_acc did not improve from 0.50418
Epoch 4/15
1944/1944 [==============================] - 56s 29ms/step - loss: 0.6933 - acc: 0.5001 - val_loss: 0.6932 - val_acc: 0.4958

Epoch 00004: val_acc did not improve from 0.50418
Epoch 5/15
1944/1944 [==============================] - 56s 29ms/step - loss: 0.6932 - acc: 0.5005 - val_loss: 0.6931 - val_acc: 0.5042

Epoch 00005: val_acc did not improve from 0.5041

In [74]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

1531/1531 [==============================] - 14s 9ms/step - loss: 0.6933 - acc: 0.4977

 테스트 정확도: 0.4977


망해서 다시해봄
sp의 단어사이즈 1.8만에서 3만으로 늘림

In [79]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=30000 --model_type=bpe --max_sentence_length=9999')
#vocab 생성이 완료되면 naver.model, naver.vocab 파일 두개가 생성 됩니다. .vocab 에서 학습된 subwords를 확인할 수 있습니다.

In [80]:
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)

sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [81]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])
x_train = np.array(train_data['document'])
x_test = np.array(test_data['document'])

In [82]:
def sp_tokenize(s, corpus):

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./naver.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({idx:word})
        index_word.update({word:idx})

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word



my_corpus = x_train
tensor, word_index, index_word = sp_tokenize(sp, my_corpus)
x_train = tensor
print(x_train)

my_corpus = x_test
tensor, word_index, index_word = sp_tokenize(sp, my_corpus)
x_test = tensor
print(x_test)

[[    7  1030     3 ...     0     0     0]
 [ 1611     8  4907 ...     0     0     0]
 [   23 28495     0 ...     0     0     0]
 ...
 [  244   130 28490 ...     0     0     0]
 [ 2007   563  5265 ...     0     0     0]
 [  227     5 10842 ...     0     0     0]]
[[ 1215   445     0 ...     0     0     0]
 [ 1375     6    70 ...     0     0     0]
 [ 2611 28464  5574 ...     0     0     0]
 ...
 [16173   821 24406 ...     0     0     0]
 [  778 18056    38 ...     0     0     0]
 [21816   254  3766 ...     0     0     0]]


In [83]:

model = Sequential()
model.add(Embedding(30000, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)



Epoch 1/15
1944/1944 [==============================] - 66s 33ms/step - loss: 0.6937 - acc: 0.5002 - val_loss: 0.6931 - val_acc: 0.5042

Epoch 00001: val_acc improved from -inf to 0.50418, saving model to best_model.h5
Epoch 2/15
1944/1944 [==============================] - 62s 32ms/step - loss: 0.6933 - acc: 0.5026 - val_loss: 0.6931 - val_acc: 0.5042

Epoch 00002: val_acc did not improve from 0.50418
Epoch 3/15
1944/1944 [==============================] - 61s 31ms/step - loss: 0.6932 - acc: 0.4977 - val_loss: 0.6931 - val_acc: 0.5042

Epoch 00003: val_acc did not improve from 0.50418
Epoch 4/15
1944/1944 [==============================] - 58s 30ms/step - loss: 0.6932 - acc: 0.4993 - val_loss: 0.6931 - val_acc: 0.5042

Epoch 00004: val_acc did not improve from 0.50418
Epoch 5/15
1944/1944 [==============================] - 60s 31ms/step - loss: 0.6932 - acc: 0.5001 - val_loss: 0.6931 - val_acc: 0.5042

Epoch 00005: val_acc did not improve from 0.50418
Epoch 6/15
1944/1944 [===========

In [84]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

1531/1531 [==============================] - 12s 8ms/step - loss: 0.6932 - acc: 0.4977

 테스트 정확도: 0.4977


학습 다 안 됐지만 이미 망한거 같아서 다시 해봄
단어장 사이즈 5000으로, 모델을 unigram

In [85]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver3 --vocab_size=5000 --model_type=unigram --max_sentence_length=9999')
#vocab 생성이 완료되면 naver3.model, naver3.vocab 파일 두개가 생성 됩니다. .vocab 에서 학습된 subwords를 확인할 수 있습니다.

In [86]:
vocab_list = pd.read_csv('naver3.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)

sp = spm.SentencePieceProcessor()
vocab_file = "naver3.model"
sp.load(vocab_file)

True

In [87]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])
x_train = np.array(train_data['document'])
x_test = np.array(test_data['document'])

In [88]:
my_corpus = x_train
tensor, word_index, index_word = sp_tokenize(sp, my_corpus)
x_train = tensor
print(x_train)

my_corpus = x_test
tensor, word_index, index_word = sp_tokenize(sp, my_corpus)
x_test = tensor
print(x_test)

[[  52 1018    6 ...    0    0    0]
 [2041   10  603 ...    0    0    0]
 [  25  374    0 ...    0    0    0]
 ...
 [ 176  152   47 ...    0    0    0]
 [2204  760  114 ...    0    0    0]
 [ 270    7 4076 ...    0    0    0]]
[[2678  369    0 ...    0    0    0]
 [1438   27   94 ...    0    0    0]
 [2285   12  576 ...    0    0    0]
 ...
 [2502   11  796 ...    0    0    0]
 [ 699 1538   12 ...    0    0    0]
 [2124   12  236 ...    0    0    0]]


In [90]:
model = Sequential()
model.add(Embedding(5000, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
1944/1944 [==============================] - 61s 30ms/step - loss: 0.6936 - acc: 0.4992 - val_loss: 0.6932 - val_acc: 0.4958

Epoch 00001: val_acc improved from -inf to 0.49582, saving model to best_model.h5
Epoch 2/15
1944/1944 [==============================] - 58s 30ms/step - loss: 0.6935 - acc: 0.5001 - val_loss: 0.6931 - val_acc: 0.5042

Epoch 00002: val_acc improved from 0.49582 to 0.50418, saving model to best_model.h5
Epoch 3/15
1944/1944 [==============================] - 58s 30ms/step - loss: 0.6936 - acc: 0.4969 - val_loss: 0.6931 - val_acc: 0.5042

Epoch 00003: val_acc did not improve from 0.50418
Epoch 4/15
1944/1944 [==============================] - 58s 30ms/step - loss: 0.6932 - acc: 0.4998 - val_loss: 0.6931 - val_acc: 0.5042

Epoch 00004: val_acc did not improve from 0.50418
Epoch 5/15
  83/1944 [>.............................] - ETA: 51s - loss: 0.6932 - acc: 0.4966

KeyboardInterrupt: 

또 학습 멈추길래 그냥 중지시킴
이번엔 char

In [93]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver4 --vocab_size=8000 --model_type=char --max_sentence_length=9999')
#vocab 생성이 완료되면 naver3.model, naver3.vocab 파일 두개가 생성 됩니다. .vocab 에서 학습된 subwords를 확인할 수 있습니다.

In [94]:
vocab_list = pd.read_csv('naver4.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)

sp = spm.SentencePieceProcessor()
vocab_file = "naver4.model"
sp.load(vocab_file)

True

In [95]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])
x_train = np.array(train_data['document'])
x_test = np.array(test_data['document'])

In [96]:
my_corpus = x_train
tensor, word_index, index_word = sp_tokenize(sp, my_corpus)
x_train = tensor
print(x_train)

my_corpus = x_test
tensor, word_index, index_word = sp_tokenize(sp, my_corpus)
x_test = tensor
print(x_test)

[[  3  15   3 ...   0   0   0]
 [  3 669   4 ...   0   0   0]
 [  3  72  41 ...   0   0   0]
 ...
 [  3   5  27 ...   0   0   0]
 [  3 248 576 ...   0   0   0]
 [  3  19 135 ...   0   0   0]]
[[  3 579   3 ...   0   0   0]
 [  3 196  82 ...   0   0   0]
 [  3   9 139 ...   0   0   0]
 ...
 [  3  37 363 ...   0   0   0]
 [  3 238  47 ...   0   0   0]
 [  3  49  41 ...   0   0   0]]


In [97]:
model = Sequential()
model.add(Embedding(5000, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
1944/1944 [==============================] - 62s 31ms/step - loss: 0.6937 - acc: 0.5028 - val_loss: 0.6616 - val_acc: 0.5585

Epoch 00001: val_acc improved from -inf to 0.55851, saving model to best_model.h5
Epoch 2/15
1944/1944 [==============================] - 60s 31ms/step - loss: 0.6606 - acc: 0.6038 - val_loss: 0.5961 - val_acc: 0.7167

Epoch 00002: val_acc improved from 0.55851 to 0.71666, saving model to best_model.h5
Epoch 3/15
1944/1944 [==============================] - 60s 31ms/step - loss: 0.5014 - acc: 0.7558 - val_loss: 0.4367 - val_acc: 0.7916

Epoch 00003: val_acc improved from 0.71666 to 0.79160, saving model to best_model.h5
Epoch 4/15
1944/1944 [==============================] - 60s 31ms/step - loss: 0.4140 - acc: 0.8089 - val_loss: 0.4193 - val_acc: 0.8073

Epoch 00004: val_acc improved from 0.79160 to 0.80735, saving model to best_model.h5
Epoch 5/15
1944/1944 [==============================] - 59s 31ms/step - loss: 0.3823 - acc: 0.8277 - val_loss: 0.39

In [98]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

1531/1531 [==============================] - 18s 12ms/step - loss: 0.3562 - acc: 0.8484

 테스트 정확도: 0.8484


vocab_size=8000 --model_type=char 로 했더니 결과가 꽤 괜찮은 것 같다!  
이제 센텐스피스 없이 학습했을 때랑 비교...

In [99]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [100]:
data_path = os.path.dirname(os.path.abspath('__file__')) + r'/'
train_data = pd.read_table(data_path + 'ratings_train.txt')
test_data = pd.read_table(data_path + 'ratings_test.txt')

In [101]:
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data.drop_duplicates(subset=['document'], inplace=True)

train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 .!?]","")
# 한글과 공백을 제외하고 모두 제거 + 온점 + ! + ?
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 .!?]","")
# 한글과 공백을 제외하고 모두 제거 + 온점 + ! + ?
test_data['document'] = test_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
test_data['document'].replace('', np.nan, inplace=True)
print(test_data.isnull().sum())

train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인
test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(test_data.isnull().values.any()) # Null 값이 존재하는지 확인

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


id            0
document    406
label         0
dtype: int64
id            0
document    169
label         0
dtype: int64
False
False


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':


In [102]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [103]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']

In [ ]:
X_train = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)
print(X_train[:3])

In [ ]:
X_test = []
for sentence in test_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
print(tokenizer.word_index)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:
max_len = 30
below_threshold_len(max_len, X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))